# Section III. MOTION PLANNING

# Chapter 12. Advanced Topics in Planning


Multi-agent planning
--------------------

Computational complexity (NP hard on graph)

Deadlock

Coordination graph

Manipulation planning and locomotion planning
---------------------------------------------

Duality between manipulation and locomotion

Planning on closed chain manifolds

Grasp planning.

Multi-modal planning. Transitions

Task and motion planning
------------------------

AI planning and scheduling

Semantics

Knowledge base
